In [2]:
from ollama import chat
from ollama import ChatResponse

response: ChatResponse = chat(model='llama3.2', messages=[
  {
    'role': 'user',
    'content': 'Why is the sky blue?',
  },
])
print(response['message']['content'])
# or access fields directly from the response object
print(response.message.content)

The sky appears blue because of a phenomenon called Rayleigh scattering, named after the British physicist Lord Rayleigh. He discovered that when sunlight enters Earth's atmosphere, it encounters tiny molecules of gases such as nitrogen and oxygen.

Here's what happens:

1. Sunlight, which is made up of different colors (known as a spectrum), passes through the atmosphere.
2. The shorter wavelengths of light, such as blue and violet, are scattered more than the longer wavelengths by the tiny molecules in the air.
3. This scattering effect occurs because the smaller molecules are more effective at deflecting the shorter wavelengths of light.

As a result, when we look up at the sky, we see mostly the blue light that has been scattered in all directions by the atmosphere. The other colors, like red and orange, are less visible to us because they have longer wavelengths that are not as easily scattered.

However, it's worth noting that the color of the sky can change depending on the time

 ref - https://medium.com/@lifanov.a.v/integrating-langgraph-with-ollama-for-advanced-llm-applications-d6c10262dafa

In [3]:
import operator
from datetime import datetime
from typing import Annotated, TypedDict, Union

from dotenv import load_dotenv
from langchain import hub
from langchain.agents import create_react_agent
from langchain_community.chat_models import ChatOllama
from langchain_core.agents import AgentAction, AgentFinish
from langchain_core.messages import BaseMessage
from langchain_core.tools import tool
from langgraph.graph import END, StateGraph
from langgraph.prebuilt import ToolExecutor, ToolInvocation

load_dotenv()

@tool
def get_now(format: str = "%Y-%m-%d %H:%M:%S"):
    """
    Get the current time
    """
    return datetime.now().strftime(format)


tools = [get_now]

tool_executor = ToolExecutor(tools)

/var/folders/6k/m4v5sk1n0d150v30knjkwb240000gn/T/ipykernel_14040/516121426.py:27: LangGraphDeprecationWarning: ToolExecutor is deprecated as of version 0.2.0 and will be removed in 0.3.0. Use langgraph.prebuilt.ToolNode instead.
  tool_executor = ToolExecutor(tools)


In [9]:
class AgentState(TypedDict):
    input: str
    chat_history: list[BaseMessage]
    agent_outcome: Union[AgentAction, AgentFinish, None]
    intermediate_steps: Annotated[list[tuple[AgentAction, str]], operator.add]


model = ChatOllama(model="llama3.2")
prompt = hub.pull("hwchase17/react")


agent_runnable = create_react_agent(model, tools, prompt)

/Users/namrateshshrivastav/miniconda3/envs/llm/lib/python3.10/site-packages/langsmith/client.py:261: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [10]:
def execute_tools(state):
    print("Called `execute_tools`")
    messages = [state["agent_outcome"]]
    last_message = messages[-1]

    tool_name = last_message.tool

    print(f"Calling tool: {tool_name}")

    action = ToolInvocation(
        tool=tool_name,
        tool_input=last_message.tool_input,
    )
    response = tool_executor.invoke(action)
    return {"intermediate_steps": [(state["agent_outcome"], response)]}


def run_agent(state):
    """
    #if you want to better manages intermediate steps
    inputs = state.copy()
    if len(inputs['intermediate_steps']) > 5:
        inputs['intermediate_steps'] = inputs['intermediate_steps'][-5:]
    """
    agent_outcome = agent_runnable.invoke(state)
    return {"agent_outcome": agent_outcome}


def should_continue(state):
    messages = [state["agent_outcome"]]
    last_message = messages[-1]
    if "Action" not in last_message.log:
        return "end"
    else:
        return "continue"

In [11]:
workflow = StateGraph(AgentState)

workflow.add_node("agent", run_agent)
workflow.add_node("action", execute_tools)


workflow.set_entry_point("agent")

workflow.add_conditional_edges(
    "agent", should_continue, {"continue": "action", "end": END}
)


workflow.add_edge("action", "agent")
app = workflow.compile()

input_text = "Whats the current time?"

In [12]:
inputs = {"input": input_text, "chat_history": []}
results = []
for s in app.stream(inputs):
    result = list(s.values())[0]
    results.append(result)
    print(result)

{'agent_outcome': AgentAction(tool='get_now', tool_input='', log='Action: get_now\nAction Input: ')}
Called `execute_tools`
Calling tool: get_now
{'intermediate_steps': [(AgentAction(tool='get_now', tool_input='', log='Action: get_now\nAction Input: '), '')]}


/var/folders/6k/m4v5sk1n0d150v30knjkwb240000gn/T/ipykernel_14040/254441368.py:10: LangGraphDeprecationWarning: ToolInvocation is deprecated as of version 0.2.0 and will be removed in 0.3.0. Use langgraph.prebuilt.ToolNode instead.
  action = ToolInvocation(


{'agent_outcome': AgentAction(tool='get_now', tool_input='', log='Question: Whats the current time?\nThought: Action: get_now\nAction Input: ')}
Called `execute_tools`
Calling tool: get_now
{'intermediate_steps': [(AgentAction(tool='get_now', tool_input='', log='Question: Whats the current time?\nThought: Action: get_now\nAction Input: '), '')]}


/var/folders/6k/m4v5sk1n0d150v30knjkwb240000gn/T/ipykernel_14040/254441368.py:10: LangGraphDeprecationWarning: ToolInvocation is deprecated as of version 0.2.0 and will be removed in 0.3.0. Use langgraph.prebuilt.ToolNode instead.
  action = ToolInvocation(


{'agent_outcome': AgentAction(tool='get_now', tool_input='%Y-%m-%d %H:%M:%S', log='Question: Whats the current time?\nThought: Action: get_now\nAction Input: %Y-%m-%d %H:%M:%S')}
Called `execute_tools`
Calling tool: get_now
{'intermediate_steps': [(AgentAction(tool='get_now', tool_input='%Y-%m-%d %H:%M:%S', log='Question: Whats the current time?\nThought: Action: get_now\nAction Input: %Y-%m-%d %H:%M:%S'), '2024-12-15 13:32:31')]}


...

/var/folders/6k/m4v5sk1n0d150v30knjkwb240000gn/T/ipykernel_14040/254441368.py:10: LangGraphDeprecationWarning: ToolInvocation is deprecated as of version 0.2.0 and will be removed in 0.3.0. Use langgraph.prebuilt.ToolNode instead.
  action = ToolInvocation(


{'agent_outcome': AgentAction(tool='get_now', tool_input='%Y-%m-%d %H:%M:%S', log='Question: Whats the current time?\nThought: Action: get_now\nAction Input: %Y-%m-%d %H:%M:%S')}
Called `execute_tools`
Calling tool: get_now
{'intermediate_steps': [(AgentAction(tool='get_now', tool_input='%Y-%m-%d %H:%M:%S', log='Question: Whats the current time?\nThought: Action: get_now\nAction Input: %Y-%m-%d %H:%M:%S'), '2024-12-15 13:32:47')]}


/var/folders/6k/m4v5sk1n0d150v30knjkwb240000gn/T/ipykernel_14040/254441368.py:10: LangGraphDeprecationWarning: ToolInvocation is deprecated as of version 0.2.0 and will be removed in 0.3.0. Use langgraph.prebuilt.ToolNode instead.
  action = ToolInvocation(


{'agent_outcome': AgentAction(tool='get_now', tool_input='%Y-%m-%d %H:%M:%S', log='Question: Whats the current time?\nThought:Action: get_now\nAction Input: %Y-%m-%d %H:%M:%S')}


GraphRecursionError: Recursion limit of 25 reached without hitting a stop condition. You can increase the limit by setting the `recursion_limit` config key.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/GRAPH_RECURSION_LIMIT